In [ ]:
import os
import json
import re
import sys
import logging
from datetime import datetime
from typing import Dict, Any, Optional, List
from dataclasses import dataclass, asdict
from collections import defaultdict
import uuid
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
from tenacity import retry, stop_after_attempt, wait_fixed

In [2]:


# Set up logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


In [ ]:

llm_config = {
    "config_list": [
        {
            "model": "gpt-4o",
            "api_key": "",
            "base_url": "",
            "api_type": "azure",
            "api_version": "2024-02-01"
        }
    ],
    "temperature": 0.7,
    "timeout": 120,
    "cache_seed": 42
}


In [ ]:
# Create directories
os.makedirs("data/conversation_history", exist_ok=True)
os.makedirs("data/progress_data", exist_ok=True)

In [ ]:


# Define user role constant
USER_ROLE = "user"


In [ ]:

# Define user role constant
USER_ROLE = "user"

# Valid difficulty levels
VALID_DIFFICULTIES = {"easy", "medium", "hard"}

# Agent Configuration
class AgentConfig:
    @staticmethod
    def get_tutor_system_message() -> str:
        return """You are an expert Educational Tutor Agent. Explain concepts clearly for high-school students, provide step-by-step breakdowns, examples, and practice problems. Evaluate student solutions with detailed feedback, identifying correctness, errors, and improvement tips. Use visual aids for visual learners when specified. Maintain a supportive tone."""

    @staticmethod
    def get_student_system_message() -> str:
        return """You are a high-school Student Agent. Forward user inputs to the appropriate agent (tutor, problem solver, or progress tracker) and engage actively by:
- Sharing user-provided problem-solving steps, questions, or solutions.
- Reflecting on feedback (e.g., 'I understand my mistake in factoring now!').
- Summarizing recent activities if asked for progress (e.g., 'I worked on quadratic equations')."""

    @staticmethod
    def get_problem_solver_system_message() -> str:
        return """You are a Problem Solver Agent. When practice problems are generated, prompt the student to solve each problem one by one, collect their solutions, and forward them to the tutor for evaluation. Display the tutor's feedback clearly. Maintain an encouraging tone and guide the student through the process."""

    @staticmethod
    def get_progress_tracker_system_message() -> str:
        return """You are a Progress Tracker Agent. Generate student-friendly JSON progress reports using ProgressMemory data. Include a summary, strengths (skill_level > 0.7), areas for improvement (skill_level < 0.5), and recommendations. Handle minimal data gracefully."""

# Conversation Memory (LangChain)
class LangChainConversationMemory:
    def __init__(self, session_id: str, storage_path: str = "data/conversation_history"):
        self.session_id = session_id
        self.storage_path = storage_path
        self.memory = ConversationBufferMemory(return_messages=True)
        self.metadata = {
            "session_id": session_id,
            "created_at": datetime.now().isoformat(),
            "subjects": [],
            "topics": []
        }
        os.makedirs(self.storage_path, exist_ok=True)
        self._load()

    def add_message(self, role: str, content: str, metadata: Optional[Dict[str, str]] = None):
        if role == USER_ROLE:
            self.memory.chat_memory.add_message(HumanMessage(content=content))
        else:
            self.memory.chat_memory.add_message(AIMessage(content=content))
        if metadata:
            if "subject" in metadata and metadata["subject"] not in self.metadata["subjects"]:
                self.metadata["subjects"].append(metadata["subject"])
            if "topic" in metadata and metadata["topic"] not in self.metadata["topics"]:
                self.metadata["topics"].append(metadata["topic"])
        self._save()

    def get_messages(self) -> List[Dict[str, Any]]:
        return [
            {"role": "human" if isinstance(msg, HumanMessage) else "assistant", "content": msg.content}
            for msg in self.memory.chat_memory.messages
        ]

    def get_context(self) -> str:
        return self.memory.buffer

    def _save(self):
        filepath = os.path.join(self.storage_path, f"{self.session_id}.json")
        try:
            data = {
                "metadata": self.metadata,
                "messages": [
                    {"role": "human" if isinstance(msg, HumanMessage) else "assistant", "content": msg.content}
                    for msg in self.memory.chat_memory.messages
                ]
            }
            with open(filepath, "w", encoding="utf-8") as f:
                json.dump(data, f, indent=2)
        except Exception as e:
            logger.error(f"Failed to save conversation: {e}")

    def _load(self):
        filepath = os.path.join(self.storage_path, f"{self.session_id}.json")
        if os.path.exists(filepath):
            try:
                with open(filepath, "r", encoding="utf-8") as f:
                    data = json.load(f)
                self.metadata = data.get("metadata", self.metadata)
                for msg in data.get("messages", []):
                    if msg["role"] == "human":
                        self.memory.chat_memory.add_message(HumanMessage(content=msg["content"]))
                    else:
                        self.memory.chat_memory.add_message(AIMessage(content=msg["content"]))
            except Exception as e:
                logger.error(f"Failed to load conversation: {e}")

# Progress Memory
@dataclass
class LearningProgress:
    subject: str
    topic: str
    skill_level: float
    attempts: int
    successful_attempts: int

@dataclass
class LearningSession:
    session_id: str
    start_time: str
    subjects_covered: List[str]
    questions_asked: int
    questions_answered_correctly: int

class ProgressMemory:
    def __init__(self, student_id: str, storage_path: str = "data/progress_data"):
        self.student_id = student_id
        self.storage_path = storage_path
        self.progress_data: Dict[str, Dict[str, LearningProgress]] = defaultdict(dict)
        self.learning_sessions: List[LearningSession] = []
        os.makedirs(self.storage_path, exist_ok=True)
        self._load()

    def start_session(self, session_id: str) -> LearningSession:
        session = LearningSession(
            session_id=session_id,
            start_time=datetime.now().isoformat(),
            subjects_covered=[],
            questions_asked=0,
            questions_answered_correctly=0
        )
        self.learning_sessions.append(session)
        self._save()
        return session

    def update_progress(self, subject: str, topic: str, performance_score: float, was_successful: bool):
        performance_score = max(0.0, min(1.0, performance_score))  # Validate score
        if subject not in self.progress_data:
            self.progress_data[subject] = {}
        if topic not in self.progress_data[subject]:
            self.progress_data[subject][topic] = LearningProgress(subject, topic, 0.0, 0, 0)
        progress = self.progress_data[subject][topic]
        progress.attempts += 1
        if was_successful:
            progress.successful_attempts += 1
        progress.skill_level = (progress.skill_level * 0.7) + (performance_score * 0.3)
        logger.info(f"Updated progress: {subject}/{topic}, skill={progress.skill_level}")
        self._save()

    def update_session(self, session_id: str, subject: str, topic: str, question_asked: bool = False, correct_answer: bool = False):
        for session in self.learning_sessions:
            if session.session_id == session_id:
                if subject not in session.subjects_covered:
                    session.subjects_covered.append(subject)
                if question_asked:
                    session.questions_asked += 1
                if correct_answer:
                    session.questions_answered_correctly += 1
                logger.info(f"Updated session {session_id}: subject={subject}, questions_asked={session.questions_asked}")
                self._save()
                break

    def get_progress_report(self) -> Dict[str, Any]:
        return {
            "student_id": self.student_id,
            "subjects": {s: {t: asdict(p) for t, p in topics.items()} for s, topics in self.progress_data.items()},
            "sessions": [asdict(s) for s in self.learning_sessions],
            "timestamp": datetime.now().isoformat()
        }

    def _save(self):
        filepath = os.path.join(self.storage_path, f"{self.student_id}_progress.json")
        try:
            data = {
                "student_id": self.student_id,
                "progress_data": {
                    s: {t: asdict(p) for t, p in topics.items()}
                    for s, topics in self.progress_data.items()
                },
                "learning_sessions": [asdict(s) for s in self.learning_sessions]
            }
            with open(filepath, "w", encoding="utf-8") as f:
                json.dump(data, f, indent=2)
        except Exception as e:
            logger.error(f"Failed to save progress: {e}")

    def _load(self):
        filepath = os.path.join(self.storage_path, f"{self.student_id}_progress.json")
        if os.path.exists(filepath):
            try:
                with open(filepath, "r", encoding="utf-8") as f:
                    data = json.load(f)
                for subject, topics in data.get("progress_data", {}).items():
                    self.progress_data[subject] = {
                        topic: LearningProgress(**progress)
                        for topic, progress in topics.items()
                    }
                self.learning_sessions = [
                    LearningSession(**session)
                    for session in data.get("learning_sessions", [])
                ]
            except Exception as e:
                logger.error(f"Failed to load progress: {e}")

# Current Problems Storage
current_problems: Dict[str, List[Dict[str, Any]]] = defaultdict(list)

# Create Agents
def create_educational_agents(student_id: str, session_id: str):
    progress_memory = ProgressMemory(student_id)
    conv_memory = LangChainConversationMemory(session_id)
    progress_memory.start_session(session_id=session_id)

    @retry(
        stop=stop_after_attempt(3),
        wait=wait_fixed(10),
        retry=retry_if_exception_type((requests.exceptions.HTTPError, requests.exceptions.RequestException)),
        before_sleep=lambda retry_state: logger.info(f"Retrying API call: attempt {retry_state.attempt_number}")
    )
    def create_practice_problems(subject: str, topic: str, count: int = 3, difficulty: str = "medium") -> List[Dict[str, Any]]:
        if count < 1 or count > 5:
            logger.error(f"Invalid problem count: {count}")
            return [{"error": "Problem count must be between 1 and 5"}]
        if difficulty.lower() not in VALID_DIFFICULTIES:
            logger.error(f"Invalid difficulty: {difficulty}")
            return [{"error": f"Difficulty must be one of {', '.join(VALID_DIFFICULTIES)}"}]
        context = conv_memory.get_context()
        prompt = f"""Using this conversation history:
{context}

Generate {count} practice problem(s) for {topic} in {subject} at {difficulty} difficulty. Include:
- One factoring problem
- One quadratic formula problem
- One word problem (if count >= 3)
For each problem, include:
- Question (e.g., 'Solve x^2 - 7x + 10 = 0')
- Correct answer (e.g., 'x = 2, x = 5')
- Solution (step-by-step explanation)
Return as JSON list, each with keys: question, correct_answer, solution."""
        try:
            response = tutor.generate_reply([{"content": prompt, "role": USER_ROLE}])
            try:
                problems = json.loads(response.strip())
            except json.JSONDecodeError:
                logger.error(f"Invalid JSON response for practice problems: {response}")
                return [{"error": "Failed to parse API response as JSON"}]
            if not isinstance(problems, list):
                problems = [problems]
            valid_problems = [
                p for p in problems if isinstance(p, dict) and all(k in p for k in ["question", "correct_answer", "solution"])
            ]
            if not valid_problems:
                logger.error("No valid problems generated")
                return [{"error": "No valid problems generated"}]
            current_problems[session_id] = [
                {
                    "subject": subject,
                    "topic": topic,
                    "problem": p["question"],
                    "correct_answer": p["correct_answer"],
                    "solution": p["solution"]
                }
                for p in valid_problems
            ]
            conv_memory.add_message("assistant", json.dumps(valid_problems, indent=2), {"subject": subject, "topic": topic})
            progress_memory.update_session(session_id, subject, topic, question_asked=True)
            logger.info(f"Generated {len(valid_problems)} practice problems for session {session_id}")
            return valid_problems
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                logger.error("Rate limit exceeded")
                raise
            elif e.response.status_code == 401:
                logger.error("Authentication error")
                raise
            logger.error(f"Failed to generate practice problems: {e}")
            return [{"error": str(e)}]
        except Exception as e:
            logger.error(f"Failed to generate practice problems: {e}")
            return [{"error": str(e)}]

    @retry(
        stop=stop_after_attempt(3),
        wait=wait_fixed(10),
        retry=retry_if_exception_type((requests.exceptions.HTTPError, requests.exceptions.RequestException)),
        before_sleep=lambda retry_state: logger.info(f"Retrying API call: attempt {retry_state.attempt_number}")
    )
    def evaluate_solution(session_id: str, problem_index: int, student_solution: str) -> Dict[str, Any]:
        if session_id not in current_problems or problem_index >= len(current_problems[session_id]):
            logger.error(f"Invalid problem index {problem_index} for session {session_id}")
            return {"error": "Invalid problem index or no problems assigned."}
        problem_data = current_problems[session_id][problem_index]
        context = conv_memory.get_context()
        prompt = f"""Using this conversation history:
{context}

Evaluate the student's solution to the following problem:
Question: {problem_data['problem']}
Correct Answer: {problem_data['correct_answer']}
Student's Solution: {student_solution}

Provide feedback in JSON format with:
- is_correct (boolean)
- feedback (detailed explanation of correctness, errors, and improvement tips)
- performance_score (float between 0.0 and 1.0)
"""
        try:
            response = tutor.generate_reply([{"content": prompt, "role": USER_ROLE}])
            try:
                evaluation = json.loads(response.strip())
            except json.JSONDecodeError:
                logger.error(f"Invalid JSON response for evaluation: {response}")
                return {"error": "Failed to parse evaluation response as JSON"}
            if not all(k in evaluation for k in ["is_correct", "feedback", "performance_score"]):
                logger.error(f"Incomplete evaluation response: {evaluation}")
                return {"error": "Incomplete evaluation response"}
            conv_memory.add_message("assistant", json.dumps(evaluation, indent=2), {"subject": problem_data["subject"], "topic": problem_data["topic"]})
            performance_score = evaluation.get("performance_score", 0.5)
            performance_score = max(0.0, min(1.0, performance_score))  # Validate score
            progress_memory.update_progress(
                problem_data["subject"],
                problem_data["topic"],
                performance_score,
                evaluation.get("is_correct", False)
            )
            progress_memory.update_session(
                session_id,
                problem_data["subject"],
                problem_data["topic"],
                question_asked=True,
                correct_answer=evaluation.get("is_correct", False)
            )
            logger.info(f"Evaluated solution for problem {problem_index} in session {session_id}")
            return evaluation
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                logger.error("Rate limit exceeded")
                raise
            elif e.response.status_code == 401:
                logger.error("Authentication error")
                raise
            logger.error(f"Failed to evaluate solution: {e}")
            return {"error": str(e)}
        except Exception as e:
            logger.error(f"Failed to evaluate solution: {e}")
            return {"error": str(e)}

    tutor = AssistantAgent(
        name="Educational_Tutor",
        system_message=AgentConfig.get_tutor_system_message(),
        llm_config=llm_config,
        human_input_mode="NEVER",
        max_consecutive_auto_reply=5
    )

    @retry(
        stop=stop_after_attempt(3),
        wait=wait_fixed(10),
        retry=retry_if_exception_type((requests.exceptions.HTTPError, requests.exceptions.RequestException)),
        before_sleep=lambda retry_state: logger.info(f"Retrying API call: attempt {retry_state.attempt_number}")
    )
    def explain_concept(subject: str, topic: str, difficulty_level: str = "medium", learning_style: str = "visual"):
        if difficulty_level.lower() not in VALID_DIFFICULTIES:
            logger.error(f"Invalid difficulty: {difficulty_level}")
            return f"Error: Difficulty must be one of {', '.join(VALID_DIFFICULTIES)}"
        context = conv_memory.get_context()
        prompt = f"""Using this conversation history:
{context}

Explain {topic} in {subject} at {difficulty_level} difficulty for a high-school student, using a {learning_style} learning style. Provide:
1. Step-by-step breakdown
2. One example
3. One practice question
"""
        try:
            response = tutor.generate_reply([{"content": prompt, "role": USER_ROLE}])
            conv_memory.add_message("assistant", response, {"subject": subject, "topic": topic})
            progress_memory.update_progress(subject, topic, 0.6, True)
            progress_memory.update_session(session_id, subject, topic, question_asked=True)
            logger.info(f"Explained concept: {subject}/{topic}")
            return response
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                logger.error("Rate limit exceeded")
                raise
            elif e.response.status_code == 401:
                logger.error("Authentication error")
                raise
            logger.error(f"Failed to explain concept: {e}")
            return f"Error: {str(e)}"
        except Exception as e:
            logger.error(f"Failed to explain concept: {e}")
            return f"Error: {str(e)}"

    tutor.register_function({
        "explain_concept": explain_concept,
        "create_practice_problems": create_practice_problems,
        "evaluate_solution": evaluate_solution
    })

    student = UserProxyAgent(
        name="Student_Learner",
        system_message=AgentConfig.get_student_system_message(),
        human_input_mode="NEVER",
        max_consecutive_auto_reply=3,
        code_execution_config=False
    )

    problem_solver = AssistantAgent(
        name="Problem_Solver",
        system_message=AgentConfig.get_problem_solver_system_message(),
        llm_config=llm_config,
        human_input_mode="NEVER",
        max_consecutive_auto_reply=3
    )

    @retry(
        stop=stop_after_attempt(3),
        wait=wait_fixed(10),
        retry=retry_if_exception_type((requests.exceptions.HTTPError, requests.exceptions.RequestException)),
        before_sleep=lambda retry_state: logger.info(f"Retrying API call: attempt {retry_state.attempt_number}")
    )
    def solve_practice_problems(session_id: str, tutor_agent: AssistantAgent):
        if session_id not in current_problems or not current_problems[session_id]:
            logger.error(f"No practice problems available for session {session_id}")
            return {"error": "No practice problems available. Request problems first."}
        responses = []
        for idx, problem_data in enumerate(current_problems[session_id]):
            print(f"\nProblem {idx + 1}: {problem_data['problem']}")
            print("Enter your solution (e.g., 'x = 2, x = 5' or describe your steps):")
            solution = input("> ").strip()
            if not solution:
                print("Solution cannot be empty. Skipping problem.")
                responses.append({"error": f"Empty solution for problem {idx + 1}"})
                continue
            conv_memory.add_message(USER_ROLE, f"Solution to problem {idx + 1}: {solution}", {"subject": problem_data["subject"], "topic": problem_data["topic"]})
            try:
                evaluation = tutor_agent.generate_reply([{
                    "content": f"Call evaluate_solution with session_id={session_id}, problem_index={idx}, student_solution={solution}",
                    "role": USER_ROLE
                }])
                try:
                    eval_data = json.loads(evaluation.strip())
                except json.JSONDecodeError:
                    logger.error(f"Invalid JSON evaluation response: {evaluation}")
                    responses.append({"error": f"Invalid evaluation for problem {idx + 1}"})
                    continue
                print(f"\nEvaluation for Problem {idx + 1}:")
                print(json.dumps(eval_data, indent=2))
                responses.append(eval_data)
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429:
                    logger.error("Rate limit exceeded")
                    raise
                elif e.response.status_code == 401:
                    logger.error("Authentication error")
                    raise
                logger.error(f"Failed to parse evaluation: {e}")
                responses.append({"error": str(e)})
            except Exception as e:
                logger.error(f"Failed to parse evaluation: {e}")
                responses.append({"error": str(e)})
        current_problems[session_id].clear()
        logger.info(f"Completed solving problems for session {session_id}")
        return responses

    problem_solver.register_function({"solve_practice_problems": lambda: solve_practice_problems(session_id, tutor)})

    progress_tracker = AssistantAgent(
        name="Progress_Tracker",
        system_message=AgentConfig.get_progress_tracker_system_message(),
        llm_config=llm_config,
        human_input_mode="NEVER"
    )

    @retry(
        stop=stop_after_attempt(3),
        wait=wait_fixed(10),
        retry=retry_if_exception_type((requests.exceptions.HTTPError, requests.exceptions.RequestException)),
        before_sleep=lambda retry_state: logger.info(f"Retrying API call: attempt {retry_state.attempt_number}")
    )
    def generate_progress_report():
        report = progress_memory.get_progress_report()
        if not report["subjects"] and not report["sessions"]:
            return {
                "summary": "No progress data available yet. Try solving problems!",
                "strengths": [],
                "areas_for_improvement": [],
                "recommendations": ["Solve practice problems", "Ask clarifying questions"],
                "timestamp": datetime.now().isoformat()
            }
        prompt = f"""Generate a student-friendly JSON progress report based on:
{json.dumps(report, indent=2)}
Include:
- Summary of progress
- Strengths (skill_level > 0.7)
- Areas for improvement (skill_level < 0.5)
- Recommendations
"""
        try:
            response = progress_tracker.generate_reply([{"content": prompt, "role": USER_ROLE}])
            try:
                report_data = json.loads(response.strip())
            except json.JSONDecodeError:
                logger.error(f"Invalid JSON response for progress report: {response}")
                return {"error": "Failed to parse progress report"}
            logger.info("Generated progress report")
            return report_data
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                logger.error("Rate limit exceeded")
                raise
            elif e.response.status_code == 401:
                logger.error("Authentication error")
                raise
            logger.error(f"Failed to generate progress report: {e}")
            return {"error": str(e)}
        except Exception as e:
            logger.error(f"Failed to generate progress report: {e}")
            return {"error": str(e)}

    progress_tracker.register_function({"generate_progress_report": generate_progress_report})

    def custom_speaker_selection(last_speaker, groupchat):
        agents = groupchat.agents
        last_message = groupchat.messages[-1]["content"].strip() if groupchat.messages else ""
        if last_speaker == student and not last_message:
            return tutor
        idx = agents.index(last_speaker) if last_speaker in agents else -1
        return agents[(idx + 1) % len(agents)]

    groupchat = GroupChat(
        agents=[student, tutor, problem_solver, progress_tracker],
        messages=[],
        max_round=20,
        speaker_selection_method=custom_speaker_selection
    )

    manager = GroupChatManager(
        groupchat=groupchat,
        llm_config=llm_config,
        name="Education_Manager"
    )

    return {
        "student": student,
        "tutor": tutor,
        "problem_solver": problem_solver,
        "progress_tracker": progress_tracker,
        "manager": manager,
        "progress_memory": progress_memory,
        "conv_memory": conv_memory,
        "create_practice_problems": create_practice_problems,
        "evaluate_solution": evaluate_solution,
        "generate_progress_report": generate_progress_report
    }

# Main Execution
def main():
    print("=== Welcome to the Educational Session ===")
    student_id = "test_student_001"
    session_id = str(uuid.uuid4())

    try:
        agents = create_educational_agents(student_id, session_id)
        student = agents["student"]
        manager = agents["manager"]
        tutor = agents["tutor"]
        problem_solver = agents["problem_solver"]
        progress_tracker = agents["progress_tracker"]
        progress_memory = agents["progress_memory"]
        conv_memory = agents["conv_memory"]
        create_practice_problems = agents["create_practice_problems"]
        evaluate_solution = agents["evaluate_solution"]
        generate_progress_report = agents["generate_progress_report"]

        print("\nAvailable Actions:")
        print("1. Explain a concept (e.g., 'Explain quadratic equations, medium difficulty, visual style')")
        print("2. Request practice problems (e.g., 'Create 3 problems on quadratic equations, medium difficulty')")
        print("3. Attempt a problem (e.g., 'For x^2 - 7x + 10 = 0, I got x = 2, x = 5. Is this correct?')")
        print("4. Work out a problem (request a single problem, solve it, and submit your solution)")
        print("5. Generate progress report")
        print("6. Exit")

        while True:
            print("\nWhat would you like to do? (Enter number 1-6)")
            action = input("> ").strip()

            if action == "6":
                print("\n=== Session Ended ===")
                break

            if action not in ["1", "2", "3", "4", "5", "6"]:
                print("Invalid action. Please choose 1-6.")
                continue

            if action == "1":
                print("Enter concept query (e.g., 'Explain quadratic equations, medium difficulty, visual style'):")
                query = input("> ").strip()
                if not query:
                    print("Query cannot be empty.")
                    continue
                subject = "Mathematics"
                topic_match = re.search(r'\b([\w\s]+)\b(?=\s*,|\s*$)', query.split(",")[0])
                topic = topic_match.group(1).strip() if topic_match else query.split()[0]
                try:
                    student.initiate_chat(manager, message=query)
                    conv_memory.add_message(USER_ROLE, query, {"subject": subject, "topic": topic})
                    progress_memory.update_progress(subject, topic, 0.6, True)
                    progress_memory.update_session(session_id, subject, topic, question_asked=True)
                except Exception as e:
                    logger.error(f"Error in concept explanation: {e}")
                    print(f"Failed to process query: {e}")

            elif action == "2":
                print("Enter practice problems query (e.g., 'Create 3 problems on quadratic equations, medium difficulty'):")
                query = input("> ").strip()
                if not query:
                    print("Query cannot be empty.")
                    continue
                subject = "Mathematics"
                topic_match = re.search(r'\b([\w\s]+)\b(?=\s*,|\s*$)', query.split(",")[0])
                topic = topic_match.group(1).strip() if topic_match else query.split()[0]
                count_match = re.search(r'\b(\d+)\b', query)
                count = int(count_match.group(1)) if count_match else 3
                difficulty = query.split(',')[1].strip() if ',' in query else "medium"
                if count < 1 or count > 5:
                    print("Number of problems must be between 1 and 5.")
                    continue
                if difficulty.lower() not in VALID_DIFFICULTIES:
                    print(f"Difficulty must be one of {', '.join(VALID_DIFFICULTIES)}.")
                    continue
                try:
                    problems = create_practice_problems(subject, topic, count, difficulty)
                    if problems and isinstance(problems[0], dict) and "error" in problems[0]:
                        logger.error(f"Problem generation failed: {problems[0]['error']}")
                        print(f"Failed to generate problems: {problems[0]['error']}")
                        continue
                    conv_memory.add_message(USER_ROLE, query, {"subject": subject, "topic": topic})
                    progress_memory.update_session(session_id, subject, topic, question_asked=True)
                    if current_problems[session_id]:
                        problem_solver.generate_reply([{
                            "content": f"Call solve_practice_problems with session_id={session_id}",
                            "role": USER_ROLE
                        }])
                    else:
                        logger.error("No problems stored after generation")
                        print("Failed to generate problems. Please try again.")
                except Exception as e:
                    logger.error(f"Error in practice problems: {e}")
                    print(f"Failed to process query: {e}")

            elif action == "3":
                print("Enter problem attempt (e.g., 'For x^2 - 7x + 10 = 0, I got x = 2, x = 5. Is this correct?'):")
                query = input("> ").strip()
                if not query:
                    print("Query cannot be empty.")
                    continue
                subject = "Mathematics"
                topic = "Quadratic Equations"
                try:
                    student.send(message=query, recipient=manager)
                    conv_memory.add_message(USER_ROLE, query, {"subject": subject, "topic": topic})
                    progress_memory.update_progress(subject, topic, 0.8, True)
                    progress_memory.update_session(session_id, subject, topic, question_asked=True, correct_answer=True)
                except Exception as e:
                    logger.error(f"Error in problem attempt: {e}")
                    print(f"Failed to process query: {e}")

            elif action == "4":
                print("Enter problem request (e.g., 'Give me one quadratic equation problem, medium difficulty'):")
                query = input("> ").strip()
                if not query:
                    print("Query cannot be empty.")
                    continue
                subject = "Mathematics"
                topic_match = re.search(r'\b([\w\s]+)\b(?=\s*,|\s*$)', query.split(",")[0])
                topic = topic_match.group(1).strip() if topic_match else query.split()[0]
                difficulty = query.split(',')[1].strip() if ',' in query else "medium"
                if difficulty.lower() not in VALID_DIFFICULTIES:
                    print(f"Difficulty must be one of {', '.join(VALID_DIFFICULTIES)}.")
                    continue
                try:
                    problems = create_practice_problems(subject, topic, count=1, difficulty=difficulty)
                    if problems and isinstance(problems[0], dict) and "error" in problems[0]:
                        logger.error(f"Problem generation failed: {problems[0]['error']}")
                        print(f"Failed to generate problem: {problems[0]['error']}")
                        continue
                    conv_memory.add_message(USER_ROLE, query, {"subject": subject, "topic": topic})
                    progress_memory.update_session(session_id, subject, topic, question_asked=True)
                    if current_problems[session_id]:
                        print(f"\nProblem: {current_problems[session_id][0]['problem']}")
                        print("Enter your solution (e.g., 'x = 2, x = 5' or describe your steps):")
                        solution = input("> ").strip()
                        if not solution:
                            print("Solution cannot be empty.")
                            continue
                        evaluation = evaluate_solution(session_id, 0, solution)
                        print("\nEvaluation:")
                        if "error" in evaluation:
                            logger.error(f"Evaluation failed: {evaluation['error']}")
                            print(f"Evaluation error: {evaluation['error']}")
                        else:
                            print(json.dumps(evaluation, indent=2))
                        current_problems[session_id].clear()
                    else:
                        logger.error("No problem stored after generation")
                        print("Failed to generate problem. Please try again.")
                except Exception as e:
                    logger.error(f"Error in single problem: {e}")
                    print(f"Failed to process query: {e}")

            elif action == "5":
                print("\n=== Generating Progress Report ===")
                try:
                    response = generate_progress_report()
                    if "error" in response:
                        logger.error(f"Progress report failed: {response['error']}")
                        print(f"Failed to generate progress report: {response['error']}")
                    else:
                        print(json.dumps(response, indent=2))
                except Exception as e:
                    logger.error(f"Error in progress report: {e}")
                    print(f"Failed to generate progress report: {e}")

        print("\n=== Session Statistics ===")
        final_report = progress_memory.get_progress_report()
        print(f"Total Sessions: {len(final_report['sessions'])}")
        print(f"Total Questions: {sum(s['questions_asked'] for s in final_report['sessions'])}")
        print(f"Subjects Covered: {[s for s in final_report['subjects'].keys()]}")

    except Exception as e:
        logger.error(f"Error in main: {e}")
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

    print("\n=== Educational Session Completed ===")

if __name__ == "__main__":
    os.environ["AUTOGEN_USE_DOCKER"] = "False"
    main()